# Train Likelihood Ratio Estimator

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_litera
ls

import logging
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

from madminer.sampling import SampleAugmenter
from madminer import sampling
from madminer.core import MadMiner
from madminer.ml import ParameterizedRatioEstimator


# MadMiner output
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

## Prepare Data Sample

In [ ]:
miner = MadMiner()
miner.load("meta/setup.h5")

sampler = SampleAugmenter('meta/delphes_data.h5')
benchmark_names = miner.benchmarks.keys()

x, theta0, theta1, y, r_xz, t_xz, n_effective = sampler.sample_train_ratio(
    theta0=sampling.benchmarks(benchmark_names),
    theta1=sampling.benchmark('benchmark_0'),
    n_samples=10000,
    folder='./meta/samples',
    filename='train_ratio'
)

_ = sampler.sample_test(
    theta=sampling.benchmark('sm'),
    n_samples=1000,
    folder='./data/samples',
    filename='test'
)
